In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error

mpc_bus = [
    [1, 51], [2, 20], [3, 39], [4, 39], [5, 0], [6, 52], [7, 19], [8, 28], [9, 0], [10, 0],
    [11, 70], [12, 47], [13, 34], [14, 14], [15, 90], [16, 25], [17, 11], [18, 60], [19, 45], [20, 18],
    [21, 14], [22, 10], [23, 7], [24, 13], [25, 0], [26, 0], [27, 71], [28, 17], [29, 24], [30, 0],
    [31, 43], [32, 59], [33, 23], [34, 59], [35, 33], [36, 31], [37, 0], [38, 0], [39, 27], [40, 66],
    [41, 37], [42, 96], [43, 18], [44, 16], [45, 53], [46, 28], [47, 34], [48, 20], [49, 87], [50, 17],
    [51, 17], [52, 18], [53, 23], [54, 113], [55, 63], [56, 84], [57, 12], [58, 12], [59, 277], [60, 78],
    [61, 0], [62, 77], [63, 0], [64, 0], [65, 0], [66, 39], [67, 28], [68, 0], [69, 0], [70, 66],
    [71, 0], [72, 12], [73, 6], [74, 68], [75, 47], [76, 68], [77, 61], [78, 71], [79, 39], [80, 130],
    [81, 0], [82, 54], [83, 20], [84, 11], [85, 24], [86, 21], [87, 0], [88, 48], [89, 0], [90, 163],
    [91, 10], [92, 65], [93, 12], [94, 30], [95, 42], [96, 38], [97, 15], [98, 34], [99, 42], [100, 37],
    [101, 22], [102, 5], [103, 23], [104, 38], [105, 31], [106, 43], [107, 50], [108, 2], [109, 8], [110, 39],
    [111, 0], [112, 68], [113, 6], [114, 8], [115, 22], [116, 184], [117, 20], [118, 33]
]
ratio_bus = np.zeros(118)
for i in range(118):
    ratio_bus[i] = mpc_bus[i][1]/sum(mpc_bus[j][1] for j in range(118))

# 读取系统参数
topo = pd.read_excel('118nodes_system.xlsx', sheet_name='topology', index_col=None, header=None) # 拓扑结构

unit = pd.read_excel('118nodes_system.xlsx', sheet_name='unit', header=0) # 每个节点机组容量、电价、上/下行备用，上/下行价格

reserve_up = 150
reserve_down = 150
nodes = 118
train_X_splits = np.array(pd.read_csv('X_train.csv', header=0))
test_X_splits = np.array(pd.read_csv('X_test.csv', header=0))
train_Y_splits = np.array(pd.read_csv('Y_train.csv', header=0))
test_Y_splits = np.array(pd.read_csv('Y_test.csv', header=0))
s = train_X_splits.shape[0]


[0.01202263 0.00471476 0.00919378 0.00919378 0.         0.01225837
 0.00447902 0.00660066 0.         0.         0.01650165 0.01107968
 0.00801509 0.00330033 0.02121641 0.00589345 0.00259312 0.01414427
 0.0106082  0.00424328 0.00330033 0.00235738 0.00165017 0.00306459
 0.         0.         0.01673739 0.00400754 0.00565771 0.
 0.01013673 0.01390853 0.00542197 0.01390853 0.00777935 0.00730787
 0.         0.         0.00636492 0.0155587  0.0087223  0.02263083
 0.00424328 0.00377181 0.01249411 0.00660066 0.00801509 0.00471476
 0.02050919 0.00400754 0.00400754 0.00424328 0.00542197 0.02663838
 0.01485149 0.01980198 0.00282885 0.00282885 0.06529939 0.01838755
 0.         0.01815182 0.         0.         0.         0.00919378
 0.00660066 0.         0.         0.0155587  0.         0.00282885
 0.00141443 0.01603017 0.01107968 0.01603017 0.01438001 0.01673739
 0.00919378 0.03064592 0.         0.01272984 0.00471476 0.00259312
 0.00565771 0.0049505  0.         0.01131542 0.         0.03842527
 0.

In [ ]:
# 建立模型
model = gp.Model()
P_c = model.addVars(unit.shape[0], s, lb=0, vtype=GRB.CONTINUOUS, name='P_c') # 火电机组出力
P_l = model.addVars(s, lb=0, vtype=GRB.CONTINUOUS, name='P_w') # 节点负荷需求
R_U = model.addVars(unit.shape[0], s, lb=0, vtype=GRB.CONTINUOUS, name='R_U') # 火电机组向上备用容量
R_D = model.addVars(unit.shape[0], s, lb=0, vtype=GRB.CONTINUOUS, name='R_D') # 火电机组向下备用容量
r_U = model.addVars(unit.shape[0], s, lb=0, vtype=GRB.CONTINUOUS, name='r_U') # 火电机组上调功率
r_D = model.addVars(unit.shape[0], s, lb=0, vtype=GRB.CONTINUOUS, name='r_D') # 火电机组下调功率
P_lsh = model.addVars(topo.shape[0], s, lb=0, vtype=GRB.CONTINUOUS, name='P_lsh') # 切负荷
theta_pl = model.addVars(train_X_splits.shape[1], lb=-1000, vtype=GRB.CONTINUOUS, name='theta_pl') # 负荷预测系数
delta_DA = model.addVars(topo.shape[0], s, lb=-180, ub=180, vtype=GRB.CONTINUOUS, name='delta_DA')
delta_RT = model.addVars(topo.shape[0], s, lb=-180, ub=180, vtype=GRB.CONTINUOUS, name='delta_RT')
ss = model.addVars(topo.shape[0], s, lb=0, vtype=GRB.CONTINUOUS, name='ss') 
b = model.addVar(lb=-10000, vtype=GRB.CONTINUOUS, name='b')

c = 9000 #切负荷成本
c_s = 1750 

# 设置目标函数
model.setObjective(gp.quicksum(R_U[i,j]*unit.iloc[i,5] + R_D[i,j]*unit.iloc[i,6] + P_c[i,j]*unit.iloc[i,2] 
                + (r_U[i,j]-r_D[i,j])*unit.iloc[i,2] for i in range(unit.shape[0]) for j in range(s))
                +gp.quicksum(P_lsh[i,j] * c for i in range(topo.shape[0]) for j in range(s))
                +gp.quicksum(ss[i,j]*c_s for i in range(topo.shape[0]) for j in range(s)))

for k in range(s):
    # 设置约束
        # 备用需求约束
    model.addConstr(gp.quicksum(R_U[i,k] for i in range(unit.shape[0])) == reserve_up)
    model.addConstr(gp.quicksum(R_D[i,k] for i in range(unit.shape[0])) == reserve_down)
        # 机组上下行备用约束
    for i in range(unit.shape[0]):
        model.addConstr(R_U[i,k] <= unit.iloc[i,3])
        model.addConstr(R_D[i,k] <= unit.iloc[i,4])
        # 日前：每个节点的功率平衡约束
    for t in range(topo.shape[0]):
            model.addConstr(P_c[t,k]-P_l[k]*ratio_bus[t]
            -gp.quicksum(topo.iloc[t,j]*(delta_DA[t,k]-delta_DA[j,k]) for j in range(topo.shape[0])) == 0)       
    model.addConstr(delta_DA[0,k] == 0)  
        # 日前：机组容量约束
    for i in range(unit.shape[0]):
        model.addConstr(P_c[i,k] <= unit.iloc[i,1]-R_U[i,k])
        model.addConstr(P_c[i,k] >= R_D[i,k])
        # 日前：负荷需求约束
    # 总和
    model.addConstr(P_l[k] == gp.quicksum(theta_pl[r] * train_X_splits[k][r] for r in range(train_X_splits.shape[1])) + b)
    model.addConstr(P_l[k] >= 0)
        # 实时：每个节点的功率平衡约束
    for t in range(topo.shape[0]):
        model.addConstr(r_U[t,k]-r_D[t,k] - ss[t,k] - train_Y_splits[k][0]*ratio_bus[t] + P_l[k]*ratio_bus[t] + P_lsh[t,k]
        -gp.quicksum(topo.iloc[t,j]*(delta_RT[t,k]-delta_RT[j,k]) - topo.iloc[t,j]*(delta_DA[t,k]-delta_DA[j,k]) for j in range(topo.shape[0]))==0)
        # 实时：备用容量约束
    for i in range(unit.shape[0]):
        model.addConstr(r_U[i,k] <= R_U[i,k])
        model.addConstr(r_D[i,k] <= R_D[i,k])
    model.addConstr(delta_RT[0,k] == 0)  
        # 添加切负荷量大于等于0的约束
    for t in range(topo.shape[0]):
        model.addConstr(P_lsh[t, k] >= 0)  # 确保切负荷量大于等于0
        # 线路容量约束
    for i in range(nodes):
        for j in range(nodes):
            if i != j and topo.iloc[i,j] < -0.1:
                model.addConstr(topo.iloc[i,j] * (delta_DA[i,k] - delta_DA[j,k]) <= 175)
                model.addConstr(topo.iloc[i,j] * (delta_DA[i,k] - delta_DA[j,k]) >= -175)
        # 线路容量约束
    for i in range(nodes):
        for j in range(nodes):
            if i != j and topo.iloc[i,j] < -0.1:
                model.addConstr(topo.iloc[i,j] * (delta_RT[i,k] - delta_RT[j,k]) <= 175)
                model.addConstr(topo.iloc[i,j] * (delta_RT[i,k] - delta_RT[j,k]) >= -175)


model.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-30
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-13400F, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4095000 rows, 1741209 columns and 10864854 nonzeros
Model fingerprint: 0x372bac48
Coefficient statistics:
  Matrix range     [5e-04, 2e+03]
  Objective range  [4e+01, 9e+03]
  Bounds range     [2e+02, 1e+04]
  RHS range        [4e-01, 8e+02]
Presolve removed 2291562 rows and 628992 columns (presolve time = 5s) ...
Presolve removed 2877966 rows and 628992 columns
Presolve time: 24.62s
Presolved: 1217034 rows, 1685517 columns, 6076980 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...


Barrier performed 0 iterations in 27.15 seconds (19.88 work units)
Optimization exhausted availabl

GurobiError: Out of memory

In [ ]:
theta_result = np.zeros(train_X_splits.shape[1])
for i in range(train_X_splits.shape[1]):
    theta_result[i] = theta_pl[i].X

test_Y_splits = pd.read_csv('Y_test.csv', header=0)

pre_Y = np.zeros(len(test_Y_splits))
for i in range(len(test_Y_splits)):
    pre_Y[i] = sum(theta_result[j]*test_X_splits[i][j] for j in range(len(theta_result)))
# 将 pre_Y 和 test_Y_splits 合并为一个 DataFrame
result_df = pd.DataFrame({
    'Actual': test_Y_splits.iloc[:, 0],  # 假设 test_Y_splits 只有一列
    'Predicted': pre_Y
})

# 将结果保存到 CSV 文件
result_df.to_csv('测试集.csv', index=False)

print("预测结果已保存到 predictions.csv")
print(result_df)

R_U 成本: 26999.999999999996
R_D 成本: 27000.0
P_c 成本: 85426.19438120529
r_U - r_D 成本: 7701.444507595379
切负荷成本: 0.0
总目标函数值: 8052991.740051248
14
[ 1.12199954e+00 -3.96652518e-01  8.24958770e+02  1.12575289e+00
  1.02777641e+00  3.26953910e+00  1.00966804e+00 -3.21876334e+00
  4.04518782e+00  1.23070426e+00  2.50294215e+00  5.18548514e+00
 -2.24588917e+01 -2.39303882e+00]
[[181 104   1  37  29  31  36  31  35  34  36  34  36  31]
 [182 112   1  36  30  33  38  32  35  36  37  34  36  31]
 [183 120   1  36  30  33  38  33  35  37  37  35  36  32]
 [184 128   1  35  31  33  38  33  35  37  38  35  36  32]
 [185 136   1  35  32  33  38  32  35  36  39  35  35  33]
 [186 144   1  34  32  32  37  31  34  36  37  34  33  33]
 [187 152   1  35  29  31  35  31  31  34  35  33  32  29]
 [188 160   1  35  28  31  35  31  30  34  31  33  30  29]
 [189 168   1  37  27  29  35  30  30  30  31  31  30  29]
 [190 176   1  35  27  30  32  30  30  31  30  32  30  29]
 [191 184   1  35  27  30  31  30  30  3